In [61]:
import numpy as np
from dataclasses import dataclass

#### Process 1

In [64]:
#class for the process 1

@dataclass
class Process1:
    @dataclass
    class State:
        price: int
    level_param: int
    alpha1: float= 0.25

    def up_prob(self,state:State)->float:
        return 1./ (1+ np.exp(-self.alpha1*(self.level_param-state.price)))

    def next_state(self,state:State)->State:
        up_move:int= np.random.binomial(1,self.up_prob(state),)[0]
        return Process1.State(price=state.price+up_move*2-1)

In [65]:
#Generator function to generate sampling traces

def simulation(process, start_state):
    state= start_state
    while True:
        yield state
        state= process.next_state(state)

In [66]:
# Generate num_traces(no. of samples) over time_steps (no. of time steps)

import itertools

def process1_price_traces(
    start_price: int,
    level_param: int,
    alpha1: float,
    time_steps: int,
    num_traces: int
) -> np.ndarray:
    process= Process1(level_param=level_param, alpha1=alpha1)
    start_state= Process1.Sate(price=start_price)
    return np.vstack([
        np.fromiter((s.price for s in itertools.islice(
            simulation(process, start_state),time_steps +1 )), float) 
        for _ in range(num_traces)])

#### Process 2

In [67]:
# mapping function to map the boolean output to int output 

from typing import Mapping, Optional
handy_map: Mapping[Optional[bool],int]={True:-1,False:1, None:0}

In [68]:
#code 

@dataclass
class Process2:   #Abstract class 
    @dataclass
    class State:  #Nested abstract class
        price:int
        is_prev_move_up: Optional[bool]
    alpha2: float= 0.75

    def up_prob(self,state:State)-> float:
        return 0.5*(1+self.alpha2* handy_map[state.is_prev_move_up])
    def next_state(self,state:State)-> State:
        up_move: int= np.random.binomial(1,self.up_prob(state),1)[0]
        return Process2.State(
            price=state.price+up_move*2-1,
            is_prev_move_up=bool(up_move)
        )

In [69]:
#generation of sampling traces of the stock price 

def process2_price_traces(
    start_price:int,
    alpha2: float,
    time_steps:int,
    num_traces: int,
)-> np.ndarray:
    process= Process2(alpha2=alpha2)
    start_state= Process2.State(price=start_price, is_prev_move_up=None)
    return np.vstack([
        np.fromiter((s.price for s in itertools.islice(
            simultation(process,start_state),
            time_steps+1
        )),float) for _ in range(num_traces)
    ])

#### Process 3

In [70]:
# The probability of a up move or a down move dataclass for process3
@dataclass
class Process3:
    @dataclass
    class State:
        num_up_moves: int #attribute of the dataclass State
        num_down_moves: int #attribute of the dataclass Process3
    alpha3: float=1.0

    def up_prob(self,state:State) ->float:
        total= state.sum_up_moves+state.num_down_moves
        if total==0:
            return 0.5
        elif state.num_down_moves==0:
            return state.num_down_moves** self.alpha3
        else:
            return 1./(1+(total/state.num_down_moves-1)**self.alpha3)
        
    def next_state(self,state:State) ->State:
        up_move: int= np.random.binomial(1,self.up_prob(state),1)[0]
        return Process3.State(
            num_up_moves=state.num_up_moves+up_move,
            num_down_moves=state.num_down_moves+1-up_move
        )

In [71]:
#generating sampling traces for the process 3

def process3_price_traces(
        start_price: int,
        alpha3: float,
        time_steps:int,
        num_traces: int,
) ->np.ndarray:
    process= Process3(alpha3=alpha3)
    start_state= Process3.State(num_up_moves=0, num_down_moves=0)
    return np.vstack([
        np.fromiter((start_price.num_up_moves-s.num_down_moves 
                     for s in itertools.isslice(simulation(process,start_state),
                                                time_steps+1)),float)
        for _ in range(num_traces)
    ])

#### Markov Process Implementation 
* Separate classes for non termianl states N, terminal states T, abstract class S for all states(terminal or non terminal)

In [72]:
from abc import ABC
from dataclasses import dataclass
from typing import Generic, Callable, TypeVar

S= TypeVar('S')
X= TypeVar('X')

class State(ABC, Generic[S]):
    state: S
    def on_non_terminal(
        self,
        f: Callable[['NonTerminal[S]'],X],
        default:X
    ) -> X:
        if isinstance(self,NonTerminal):
            return f(self)
        else: 
            return default

@dataclass(frozen=True)
class Terminal(State[S]):
    state:S
@dataclass(frozen=True)
class NonTerminal(State[S]):
    state:S

##### Class reresenting Markov Process
* Create ABC MarkovProcess 

In [73]:
from abc import abstractmethod
from distribution import Distribution  